In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [20]:
url = "https://wiki.uibk.ac.at/noscemus/A_Latin_Letter_containing_some_Animadversions_upon_Mr._Isaac_Newton,_his_Theory_of_Light"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    tabulka = soup.find("table", class_="wikitable")
    data = tabulka.find_all("tr")
    print(data)
    panda_tabulka = pd.DataFrame()

    panda_tabulka = pd.DataFrame([data.find_all("td")], columns=[data.find_all("th")])
else:
    print("Smula, response code je:", response.status_code)


[<tr>
<th> Author
</th>
<td> <a href="/noscemus/Pardies,_Ignace_Gaston" title="Pardies, Ignace Gaston">Pardies, Ignace Gaston</a>
</td></tr>, <tr>
<th> Full title
</th>
<td> A Latin Letter written to the Publisher April 9. 1672. n. st. by Ignatius Gaston Pardies P. Prof. of the Mathematicks in the Parisian Colledge of Clermont, containing some Animadversions upon Mr. Isaac Newton, Prof. of the Mathematicks in the University of Cambridge, his Theory of Light, printed in N.° 80.
</td></tr>, <tr>
<th> In
</th>
<td> Philosophical Transactions of the Royal Society 7/84, 4087–90
</td></tr>, <tr>
<th> Year
</th>
<td> 1672
</td></tr>, <tr>
<th> Place
</th>
<td> London
</td></tr>, <tr>
<th> Publisher/Printer
</th>
<td> Martyn
</td></tr>, <tr>
<th> Era
</th>
<td> 17th century
</td></tr>, <tr>
<th> Form/Genre
</th>
<td> Letter, Review
</td></tr>, <tr>
<th> Discipline/Content
</th>
<td> Physics
</td></tr>, <tr>
<th colspan="2" style="text-align:left">Digital copies
</th></tr>, <tr>
<td>Original
</

In [ ]:
data = []
table = soup.find('table', attrs={'class':'lineItemsTable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values